# Persona-PeaCoK

This notebook serves as a place where the (simple) Persona-PeaCoK dataset will be adapted to the LMEDR format.

--NOTE-- - design the code here, then move it to the build_data and save_datasets file

In [ ]:
import json
import os

def reformat_sample_to_convai2(sample, use_cands=True):
    persona_prefix = 'your persona:'
    sample_convai2_format = []
    for i, line in enumerate(sample['persona1_ori'] + sample['persona1_ext']):
        sample_convai2_format.append(f"{i + 1} {persona_prefix} {line}")

    num_persona_facts = len(sample_convai2_format)
    persona_prefix = 'their persona:'
    # for i, line in enumerate(list(sample['persona2_ext'].values())[-1]):
    for i, line in enumerate(list(sample['persona2_ori'])+list(sample['persona2_ext'])):
        sample_convai2_format.append(f"{num_persona_facts + i + 1} {persona_prefix} {line}")

    num_persona_facts = len(sample_convai2_format)
    for i, line in enumerate(sample[f'text_plain{"_cands" if use_cands else ""}']):
        sample_convai2_format.append(f"{num_persona_facts + i + 1} {line}")

    return sample_convai2_format


def reformat_sample_to_convai2_with_turns(sample, use_cands=True):
    sample_convai2_format = []
    turn_num = len(sample['persona1_ext'].keys())
    # for turn in sample['persona1_ext'].keys():
    persona_prefix = 'your persona:'
    curr_turn_extended_persona = sample['persona1_ori'] + sample['persona1_ext'][f"{turn_num-1}"]
    for i, line in enumerate(curr_turn_extended_persona):
        line_ind = len(sample_convai2_format) + 1
        sample_convai2_format.append(
            f"{line_ind} {persona_prefix} {line}"
        )
        
    num_persona_facts = len(sample_convai2_format)
    persona_prefix = 'their persona:'
    # for i, line in enumerate(list(sample['persona2_ext'].values())[-1]):
    turn_num = len(sample['persona2_ext'].keys())
    curr_turn_extended_persona = sample['persona2_ext'][f"{turn_num-1}"]
    for i, line in enumerate(curr_turn_extended_persona):
        line_ind = len(sample_convai2_format) + 1
        sample_convai2_format.append(
            f"{line_ind} {persona_prefix} {line}"
            )

    num_persona_facts = len(sample_convai2_format)
    for i, line in enumerate(sample[f'text_plain{"_cands" if use_cands else ""}']):
        sample_convai2_format.append(f"{num_persona_facts + i + 1} {line}")

    return sample_convai2_format

def reformat_file_to_convai2(data_file, use_cands=True, retrieved_induced=False):
    data = list(json.load(open(data_file)).values())
    data_convai2_format = []
    for sample in data:
        if retrieved_induced:
            sample_convai2_format = reformat_sample_to_convai2_with_turns(sample, use_cands)
        else:
            sample_convai2_format = reformat_sample_to_convai2(sample, use_cands)
        data_convai2_format.extend(sample_convai2_format)

    return data_convai2_format


In [6]:
input_dir = './data/ConvAI2/retrieved_2_1/'
output_dir = './data/ConvAI2/retrieved_2_1/' # adjust the dirs as needed


for (in_filename, out_filename) in [
    ('train_persona_revised_chat_ext.json', 'train_self_revised_peacok.txt'),
    ('valid_persona_revised_chat_ext.json', 'valid_self_revised_peacok.txt'),
    ('train_persona_original_chat_ext_retrieved_induced_kmax2_ksing1.json', 'train_self_original.txt'),
    ('valid_persona_original_chat_ext_retrieved_induced_kmax2_ksing1.json', 'valid_self_original.txt'), # adjust the files as needed
    ]:
    in_filepath = os.path.join(input_dir, in_filename)
    out_filepath = os.path.join(output_dir, out_filename)

    data_convai2_format = reformat_file_to_convai2(
        in_filepath,
        use_cands=bool('no_cands' not in out_filename),
        retrieved_induced=bool('induced' in in_filename),
    )

    with open(out_filepath, 'w') as f:
        f.write('\n'.join(data_convai2_format))
